<h1> ELO Merchant Category Recommendation </h1>
<h2> Part III - Feature Engineering </h2>

In [163]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [140]:
# Basic Packages
import pandas as pd
import numpy as np
import datetime
import time
import warnings
warnings.filterwarnings('ignore')
import gc

# ML Packages
import lightgbm as lgb
from sklearn import model_selection, preprocessing, metrics

# plotting packages
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

### Train and Test Sets

In [141]:
def load_data(testing=True):
    
    print('Loading Data...')
    print(f'Testing set to {testing}')
    
    start = time.time()
    train_test_dtypes = {'feature_1':'int16',
                        'feature_2':'int16',
                        'feature_3':'int16'}
    
    hist_dtypes = {'authorized_flag':'str',
                   'card_id':'str',
                   'city_id':'int16',
                   'installments':'int16',
                   'category_3':'str',
                   'merchant_category_id':'int16',
                   'merchant_id':'str',
                   'purchase_amount':'float',
                   'state_id':'int16',
                   'subsector_id':'int16'}
    if testing:
        n = 1000000
    else:
        n=None

    print('Loading Train Set...1/4',end='\r')
    train_df = pd.read_csv("train.csv",dtype=train_test_dtypes,parse_dates=['first_active_month'],nrows=n)
    print(' '*100,end='\r',flush=True)
    print('Loading Test Set...2/4',end='\r')
    test_df = pd.read_csv("test.csv",dtype=train_test_dtypes,parse_dates=['first_active_month'],nrows=n)
    print(' '*100,end='\r',flush=True)
    print('Loading New Merchant Transactions...3/4',end='\r',flush=True)
    new_trans_df = pd.read_csv('new_merchant_transactions.csv',dtype=hist_dtypes,parse_dates=True,nrows=n)
    print(' '*100,end='\r',flush=True)
    print('Loading Merchant Transactions...4/4')
    hist_df = pd.read_csv("historical_transactions.csv",dtype=hist_dtypes,parse_dates=True,nrows=n)
    print(' '*50,end='\r')
    print('Data Successfully Loaded')
    print(f'Time Taken: {time.time()-start:.2f} seconds')
    print('-'*50)
    return train_df,test_df,new_trans_df,hist_df

In [142]:
def clean_train_test(train_df,test_df):
    fill = test_df.loc[:,'first_active_month'].mode().values[0]
    test_df['first_active_month'].fillna(fill,inplace=True)
    
    for df in [train_df,test_df]:
        df['first_active_month'] = pd.to_datetime(df['first_active_month'])
        df['month'] = df['first_active_month'].dt.month
        df['elapsed_time'] = (datetime.datetime.today() - df['first_active_month']).dt.days
        
    return train_df,test_df

def fill_all_nans(df):
    
    print('Filling NaNs...1/5',end='\r')
    # Replace Values with NaNs
    df['installments'] = df['installments'].replace(999,np.nan)
    df['installments'] = df['installments'].fillna(df['installments'].mean())
    df['installments'].fillna(df['installments'].mean(),inplace=True)

    # Fill with Mode for Categorical Columns
    fill_neg1_cols = ['city_id',
                      'merchant_category_id',
                      'state_id',
                      'subsector_id',
                      'category_3',
                      'category_2']
    
    for col in fill_neg1_cols:
        df[col] = df[col].replace(-1,np.nan)
        fill = df.loc[:,col].mode().values[0]
        df[col].fillna(fill,inplace=True)
    return df

def encode_categorical_features(df):
    print('Encoding Categorical Features...2/5',end='\r')
    # Encode Categorical Variables
    df['purchase_amount'] = np.round(df['purchase_amount'] / 0.00150265118 + 497.06,2)
    df['category_1'] = df['category_1'].map({'Y':1,'N':0}).astype('bool')
    df['authorized_flag'] = df['authorized_flag'].map({'Y':1,'N':0}).astype('bool')

    return df

def create_dt_features(df):
    print('Creating Date Time Features...3/5',end='\r')
    # Create Date Time Features
    df['purchase_date'] = pd.to_datetime(df['purchase_date'])
    df['year'] = df['purchase_date'].dt.year.astype('int16')
    df['month'] = df['purchase_date'].dt.month.astype('int16')
    df['weekofyear'] = df['purchase_date'].dt.weekofyear.astype('int16')
    df['day'] = df['purchase_date'].dt.day.astype('int16')
    df['dayofweek'] = df['purchase_date'].dt.dayofweek.astype('int16')
    df['weekend'] = (df.purchase_date.dt.weekday >=5).astype('bool')
    df['hour'] = df['purchase_date'].dt.hour.astype('int16')
    df['month_diff'] = (((datetime.datetime.today()-df['purchase_date']).dt.days)//30).astype('int16')
    df['month_diff'] += df['month_lag']
    return df

def create_additional_features(df):
    print('Creating Additional Features...4/5',end='\r')
    last_hist_date = datetime.datetime(2018,2,28)
    # Other Features
    df['time_since_purchase_date'] = (last_hist_date-df['purchase_date']).dt.days
    df['duration'] = df['purchase_amount']*df['month_diff']
    df['amount_month_ratio'] = df['purchase_amount']/df['month_diff']
    df['price'] = df['purchase_amount']/df['installments']

def extra_cleaning_steps(new_trans_df,hist_df):
    print('Performing Additional Cleaning Steps...5/5',end='\r')
    # drop authorized_flag column in new df
    new_trans_df.drop(columns = 'authorized_flag',inplace=True)

    # frequency encoding for hist_df
    store_size = hist_df.groupby('merchant_id').size()
    store_size = store_size/len(hist_df)

    city_size = hist_df.groupby('city_id').size()
    city_size = city_size/len(hist_df)

    subsector_size = hist_df.groupby('subsector_id').size()
    subsector_size = subsector_size/len(hist_df)

    state_size = hist_df.groupby('state_id').size()
    state_size = state_size/len(hist_df)

    category_size = hist_df.groupby('merchant_category_id').size()
    category_size = category_size/len(hist_df)

    hist_df['store_size'] = hist_df['merchant_id'].map(store_size)
    hist_df['city_size'] = hist_df['city_id'].map(city_size)
    hist_df['subsector_size'] = hist_df['subsector_id'].map(subsector_size)
    hist_df['state_size'] = hist_df['state_id'].map(state_size)
    hist_df['category_size'] = hist_df['merchant_category_id'].map(category_size)
    
    # One Hot Encoding for Categorical Features
    hist_df = pd.get_dummies(hist_df,columns = ['category_2','category_3'])
    new_trans_df = pd.get_dummies(new_trans_df,columns = ['category_2','category_3'])
    return new_trans_df,hist_df

def clean_transactions(new_trans_df,hist_df):
    for df in [new_trans_df,hist_df]:
        df = fill_all_nans(df)
        print(' '*50,end='\r')
        df = encode_categorical_features(df)
        print(' '*50,end='\r')
        df = create_dt_features(df)
        print(' '*50,end='\r')
        df = create_additional_features(df)
        print(' '*50,end='\r')
    new_trans_df,hist_df = extra_cleaning_steps(new_trans_df,hist_df)
    print(' '*50,end='\r')
    return new_trans_df,hist_df

def preprocess_data(train_df,test_df,new_trans_df,hist_df):
    start = time.time()
    print('Preprocessing Data...')
    train_df,test_df = clean_train_test(train_df,test_df)
    new_trans_df,hist_df = clean_transactions(new_trans_df,hist_df)
    print('Data Successfully Preprocessed')
    print(f'Time Taken: {time.time()-start:.2f} seconds')
    print('-'*50)
    return train_df,test_df,new_trans_df,hist_df   

In [143]:
train_df,test_df,new_trans_df,hist_df = load_data(testing=False)
train_df,test_df,new_trans_df,hist_df = preprocess_data(train_df,test_df,new_trans_df,hist_df)

Loading Data...
Testing set to False
Loading Merchant Transactions...4/4                                                                 
Data Successfully Loaded                          
Time Taken: 139.06 seconds
--------------------------------------------------
Preprocessing Data...
Data Successfully Preprocessed                    
Time Taken: 272.74 seconds
--------------------------------------------------


This notebook starts off where our preprocessing notebook left off. We now have a "Clean" data set and can now start thinking about how we can use the merchant transactions dataframes with our train and test sets. Here we will get aggregates of each card_id then pass those values to our train and test with a join.

In [144]:
# Aggregating Historical Transactions DataFrame by card_id

print('Aggregating Historical Transactions...')

# Create dictionary of column names and aggregation functions to use

agg_func = {'authorized_flag' : ['mean'],
            'card_id' : ['count'],
            'city_id' : ['nunique'],
            'category_1' : ['sum','mean','std'],
            'category_2_1.0':['sum','mean'],
            'category_2_2.0': ['sum', 'mean'],
            'category_2_3.0': ['sum', 'mean'],
            'category_2_4.0': ['sum', 'mean'],
            'category_2_5.0': ['sum', 'mean'],
            'category_3_A': ['sum', 'mean'],
            'category_3_B': ['sum', 'mean'],
            'category_3_C': ['sum', 'mean'],
            'month':['nunique'],
            'hour':['mean'],
            'weekofyear':['mean','nunique'],
            'day':['nunique',np.ptp,'mean'],
            'dayofweek':['mean'],
            'weekend':['sum','mean'],
            'duration':['min','mean','max'],
            'price':['sum','mean','max','min','var'],
            'amount_month_ratio':['max','min',np.ptp],
            'installments': ['sum','min','max','var','mean'],
            'merchant_category_id':['nunique'],
            'merchant_id':['nunique'],
            'purchase_amount':['sum','mean','max','min','var','median'],
            'purchase_date':['max','min',np.ptp],
            'time_since_purchase_date':['min','max','mean'],
            'month_lag':['min','max','mean','var',np.ptp],
            'month_diff':['mean','min','max',np.ptp,'var'],
            'store_size':['min','max','mean'],
            'city_size':['min','max','mean'],
            'state_size':['min','max','mean'],
            'subsector_size':['min','max','mean'],
            'category_size':['min','max','mean'],
           }

# Aggregate columns based on dictionary passed to agg function
ghist_df = hist_df.groupby(['card_id']).agg(agg_func)

# Rename columns before joining train/test set
ghist_df.columns = ['hist_'+'_'.join(col).strip() for col in ghist_df.columns.values]
ghist_df.head()

Aggregating Historical Transactions...


,hist_authorized_flag_mean,hist_card_id_count,hist_city_id_nunique,hist_category_1_sum,hist_category_1_mean,hist_category_1_std,hist_category_2_1.0_sum,hist_category_2_1.0_mean,hist_category_2_2.0_sum,hist_category_2_2.0_mean,...,hist_city_size_mean,hist_state_size_min,hist_state_size_max,hist_state_size_mean,hist_subsector_size_min,hist_subsector_size_max,hist_subsector_size_mean,hist_category_size_min,hist_category_size_max,hist_category_size_mean
card_id,,,,,,,,,,,,,,,,,,,,,
C_ID_00007093c1,0.765101,149,4,28.0,0.187919,0.391965,28.0,0.187919,0.0,0.000000,...,0.051407,0.012769,0.454163,0.095844,0.002815,0.192234,0.089125,0.000148,0.190856,0.078497
C_ID_0001238066,0.975610,123,17,2.0,0.016260,0.126992,103.0,0.837398,0.0,0.000000,...,0.050689,0.031931,0.454163,0.380550,0.002815,0.192234,0.114675,0.000171,0.190856,0.087234
C_ID_0001506ef0,0.939394,66,3,0.0,0.000000,0.000000,2.0,0.030303,0.0,0.000000,...,0.027416,0.041367,0.454163,0.053876,0.008898,0.192234,0.106659,0.000889,0.190856,0.084181
C_ID_0001793786,0.875000,216,9,2.0,0.009259,0.096001,125.0,0.578704,76.0,0.351852,...,0.018768,0.007074,0.454163,0.266820,0.000803,0.192234,0.087159,0.000009,0.190856,0.054679
C_ID_000183fdda,0.951389,144,8,4.0,0.027778,0.164909,11.0,0.076389,1.0,0.006944,...,0.024582,0.013220,0.454163,0.052191,0.001349,0.192234,0.068389,0.000153,0.190856,0.040708


In [145]:
# Aggregate Columns based on Dictionary for new_trans_df
print('Aggregating New Transactions DataFrame...')

agg_func = {
            'card_id' : ['count'],
            'city_id' : ['nunique'],
            'category_1' : ['sum','mean'],
            'category_2_1.0':['sum','mean'],
            'category_2_2.0': ['sum', 'mean'],
            'category_2_3.0': ['sum', 'mean'],
            'category_2_4.0': ['sum', 'mean'],
            'category_2_5.0': ['sum', 'mean'],
            'category_3_A': ['sum', 'mean'],
            'category_3_B': ['sum', 'mean'],
            'category_3_C': ['sum', 'mean'],
            'month':['nunique'],
            'weekofyear':['nunique'],
            'day':['nunique',np.ptp,'mean'],
            'dayofweek':['mean'],
            'duration':['min','max'],
            'price':['sum','mean','max','min','var'],
            'amount_month_ratio':['max','min',np.ptp],
            'installments': ['sum','min','max','var','mean'],
            'merchant_category_id':['nunique'],
            'merchant_id':['nunique'],
            'purchase_amount':['sum','mean','max','min','var'],
            'purchase_date':['max','min',np.ptp],
            'time_since_purchase_date':['min','max','mean'],
            'weekend':['sum','mean'],
            'month_lag':['min','max','mean','var',np.ptp],
            'month_diff':['mean','min','max',np.ptp]
           }

gnew_trans_df = new_trans_df.groupby(['card_id']).agg(agg_func)

# Rename columns before joining train / test set
gnew_trans_df.columns = ['new_'+'_'.join(col).strip() for col in gnew_trans_df.columns.values]
gnew_trans_df.head()

Aggregating New Transactions DataFrame...


,new_card_id_count,new_city_id_nunique,new_category_1_sum,new_category_1_mean,new_category_2_1.0_sum,new_category_2_1.0_mean,new_category_2_2.0_sum,new_category_2_2.0_mean,new_category_2_3.0_sum,new_category_2_3.0_mean,...,new_weekend_mean,new_month_lag_min,new_month_lag_max,new_month_lag_mean,new_month_lag_var,new_month_lag_ptp,new_month_diff_mean,new_month_diff_min,new_month_diff_max,new_month_diff_ptp
card_id,,,,,,,,,,,,,,,,,,,,,
C_ID_00007093c1,2,2,0.0,0.000000,1,0.500000,0,0.000000,1,0.50000,...,0.000000,2,2,2.000000,0.000000,0,13.000000,13,13,0
C_ID_0001238066,26,7,2.0,0.076923,23,0.884615,0,0.000000,0,0.00000,...,0.461538,1,2,1.346154,0.235385,1,12.307692,12,13,1
C_ID_0001506ef0,2,1,0.0,0.000000,0,0.000000,0,0.000000,2,1.00000,...,0.000000,1,1,1.000000,0.000000,0,12.000000,12,12,0
C_ID_0001793786,31,7,0.0,0.000000,17,0.548387,8,0.258065,5,0.16129,...,0.451613,1,2,1.322581,0.225806,1,16.193548,16,17,1
C_ID_000183fdda,11,2,0.0,0.000000,0,0.000000,0,0.000000,11,1.00000,...,0.181818,1,2,1.272727,0.218182,1,12.727273,12,13,1


The next features we add are based on how often each card_id makes the same purchase or purchases from the same merchant. This will likely give us some info on customer behavior that will help our model.

In [146]:
gdf = pd.DataFrame(hist_df.groupby(['card_id','purchase_amount']).size().max(level=0))
gdf.columns = ['max_repeat_purchases']
gdf['mean_repeat_purchases'] = hist_df.groupby(['card_id','purchase_amount']).size().mean(level=0)
gdf['mean_merchant_purchases'] = hist_df.groupby(['card_id','merchant_id']).size().mean(level=0)
gdf['mean_category_purchases'] = hist_df.groupby(['card_id','merchant_category_id']).size().mean(level=0)
gdf['mean_monthly_purchases'] = hist_df.groupby(['card_id','month_diff']).size().mean(level=0)
gdf.head(3)

,max_repeat_purchases,mean_repeat_purchases,mean_merchant_purchases,mean_category_purchases,mean_monthly_purchases
card_id,,,,,
C_ID_00007093c1,13,1.674157,5.137931,8.277778,74.5
C_ID_0001238066,9,1.182692,1.892308,4.241379,61.5
C_ID_0001506ef0,4,1.200000,2.357143,3.473684,33.0


Now that we have aggregated features from our historical transactions data frame we can join them with our train and test set.

In [156]:
# Merge with train and test set
print('Merging with training set...')
train = pd.merge(train_df,ghist_df,on='card_id',how='left')
train = pd.merge(train,gnew_trans_df,on='card_id',how='left')
train = pd.merge(train,gdf,on='card_id',how='left')

print('Merging with testing set...')
test = pd.merge(test_df,ghist_df,on='card_id',how='left')
test = pd.merge(test,gnew_trans_df,on='card_id',how='left')
test = pd.merge(test,gdf,on='card_id',how='left')

Merging with training set...
Merging with testing set...


For this particular case, card_ids not present in the new_merchant dataframe will be interprited as zeros. Therefor values indicating purchase amounts/transactions will be set to zero.

In [157]:
# Card_ids not present in new trans df were filled with zeros to represent zero purchases
zero_sum_columns = ['new_purchase_amount_mean',
                    'new_purchase_amount_sum',
                    'new_purchase_amount_max',
                    'new_duration_max',
                    'new_duration_min',
                    'new_amount_month_ratio_min',
                    'new_amount_month_ratio_max',
                    'new_card_id_count',
                   ]

for col in zero_sum_columns:
    train[col] = train[col].fillna(0)
    test[col] = test[col].fillna(0)

Next we will create some time based features from features in the original training set.

In [158]:
for df in [train,test]:
    df['hist_purchase_date_uptonow'] = (datetime.datetime.today() - 
                                      df['hist_purchase_date_max']).dt.days
    df['new_purchase_date_uptonow'] = (datetime.datetime.today() - 
                                      df['new_purchase_date_max']).dt.days
    
    df['quarter'] = df['first_active_month'].dt.quarter
    df['elapsed_time'] = (datetime.datetime.today() - df['first_active_month']).dt.days
    df['days_feature1'] = df['elapsed_time'] * df['feature_1']
    df['days_feature2'] = df['elapsed_time'] * df['feature_2']
    df['days_feature3'] = df['elapsed_time'] * df['feature_3']
    df['days_feature1_ratio'] = df['feature_1'] / df['elapsed_time']
    df['days_feature2_ratio'] = df['feature_2'] / df['elapsed_time']
    df['days_feature3_ratio'] = df['feature_3'] / df['elapsed_time']
    
    dt_features = ['hist_purchase_date_max','hist_purchase_date_min',
               'new_purchase_date_max','new_purchase_date_min','hist_purchase_date_ptp','new_purchase_date_ptp']
    
    # Models cannot use datetime features so they are encoded here as int64s
    for feature in dt_features:
        df[feature] = df[feature].astype(np.int64)*1e-9
    
    df['first_month'] = df['first_active_month'].dt.month
    df['first_year'] = df['first_active_month'].dt.year
    df.drop(columns = ['first_active_month'],inplace=True)

Finally we create feature interactions between the new and historical merchant aggregate values.

In [159]:
for df in [train,test]:
    # https://www.kaggle.com/mfjwr1/simple-lightgbm-without-blending
    df['category_1_mean'] = df['new_category_1_mean']+df['hist_category_1_mean']
    df['new_CLV'] = df['new_card_id_count'] * df['new_purchase_amount_sum'] / df['new_month_diff_mean']
    df['hist_CLV'] = df['hist_card_id_count'] * df['hist_purchase_amount_sum'] / df['hist_month_diff_mean']
        
    # Ratios
    df['transactions_ratio'] = df['new_card_id_count']/df['hist_card_id_count']
    df['mean_purchase_ratio']  = df['new_purchase_amount_mean']/df['hist_purchase_amount_mean']
    df['max_purchase_ratio'] = df['new_purchase_amount_max']/df['hist_purchase_amount_max']
    df['sum_purchase_amount_ratio'] = df['new_purchase_amount_sum'] / df['hist_purchase_amount_sum']
    df['mean_month_lag_ratio'] = df['new_month_lag_mean']/df['hist_month_lag_mean']
    df['mean_month_diff_ratio'] = df['new_month_diff_mean'] / df['hist_month_diff_mean']
    df['sum_installments_ratio'] = df['new_installments_sum'] / df['hist_installments_sum']
    df['mean_installments_ratio'] = df['new_installments_mean'] / df['hist_installments_mean']
    df['min_duration_ratio'] = df['new_duration_min']/df['hist_duration_min']

    # Products
    df['transactions_product'] = df['new_card_id_count']*df['hist_card_id_count']
    df['mean_purchase_product']  = df['new_purchase_amount_mean']*df['hist_purchase_amount_mean']
    df['max_purchase_product'] = df['new_purchase_amount_max']*df['hist_purchase_amount_max']
    df['sum_purchase_amount_product'] = df['new_purchase_amount_sum']*df['hist_purchase_amount_sum']
    df['mean_month_lag_product'] = df['new_month_lag_mean']*df['hist_month_lag_mean']
    df['mean_month_diff_product'] = df['new_month_diff_mean']*df['hist_month_diff_mean']
    df['sum_installments_product'] = df['new_installments_sum']*df['hist_installments_sum']
    df['mean_installments_product'] = df['new_installments_mean']*df['hist_installments_mean']
    df['min_duration_product'] = df['new_duration_min']*df['hist_duration_min']
    
    # Weighted Time Features
    df['hist_min_duration_weighted'] = df['hist_duration_min']*df['hist_card_id_count']
    df['hist_max_duration_weighted'] = df['hist_duration_max']*df['hist_card_id_count']
    df['hist_time_since_purchase_date_min_weighted'] = df['hist_time_since_purchase_date_min']*df['hist_card_id_count']
    
    df['new_min_duration_weighted'] = df['new_duration_min']*df['new_card_id_count']
    df['new_max_duration_weighted'] = df['new_duration_max']*df['new_card_id_count']
    df['new_time_since_purchase_date_min_weighted'] = df['new_time_since_purchase_date_min']*df['new_card_id_count']
    
    # Additional Features
    df['sum_price_ratio'] = df['new_price_sum'] / df['hist_price_sum']
    df['mean_price_ratio'] = df['new_price_mean'] / df['hist_price_mean']
    df['CLV_Ratio'] = df['new_CLV']/df['hist_CLV']

This concludes all the feature engineering done for this project. We will now start training models to evaluate performance and eliminate useless features.

In [161]:
train.to_csv('final_train.csv',index=False)
test.to_csv('final_test.csv',index=False)

---